In [17]:
import os
import torch
import numpy as np
import pandas as pd
from transformers import XLNetTokenizer, XLNetLMHeadModel
import re

In [2]:
# Verificar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [27]:
model_name = '../models/prot_xlnet_generation_finetuned'
out_dir = '../data/processed/generated_seqs.csv'

In [6]:
# Cargar tokenizador y modelo para generación de lenguaje
print("Cargando tokenizador y modelo para generación de secuencias...")
tokenizer = XLNetTokenizer.from_pretrained(model_name)
# Usar XLNetLMHeadModel para generación de texto/secuencias
model = XLNetLMHeadModel.from_pretrained(model_name)

Cargando tokenizador y modelo para generación de secuencias...


In [7]:
print(f"Vocabulario del tokenizador: {tokenizer.vocab_size} tokens")
print("El modelo está configurado para generación de secuencias de proteínas")

Vocabulario del tokenizador: 37 tokens
El modelo está configurado para generación de secuencias de proteínas


In [8]:
# Mover el modelo al dispositivo adecuado
model.to(device)

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(37, 1024)
    (layer): ModuleList(
      (0-29): 30 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): ReLU()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_loss): Linear(in_features=1024, out_features=37, bias=True)
)

In [9]:
# Funciones para generar nuevas secuencias
def generate_protein_sequence(prompt="", max_new_tokens=100, temperature=0.8, do_sample=True, repetition_penalty = 1.1):
    """
    Genera una nueva secuencia de proteína.
    
    Args:
        prompt: Secuencia inicial (puede estar vacía)
        max_new_tokens: Número máximo de tokens nuevos a generar
        temperature: Controla la aleatoriedad (más alto = más aleatorio)
        do_sample: Si usar sampling o greedy decoding
    """
    model.eval()
    
    # Si no hay prompt, usar un token especial o secuencia corta común
    if not prompt:
        prompt = "M"  # Muchas proteínas empiezan con metionina
    
    # Tokenizar el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # Generar secuencia
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty= repetition_penalty #1.1
        )
    
    # Decodificar la secuencia generada
    generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remover el prompt original para obtener solo la parte generada
    if prompt:
        generated_sequence = generated_sequence[len(prompt):]
    
    return generated_sequence

In [25]:
def generate_multiple_sequences(num_sequences=5, show_output = True, **generation_kwargs):
    """
    Genera múltiples secuencias de proteínas.
    """
    sequences = []
    for i in range(num_sequences):
        seq = generate_protein_sequence(**generation_kwargs)
        seq = re.sub(r'\s+', '', seq)
        sequences.append(seq)
        if show_output:
         print(f"Secuencia {i+1}: {seq}")
        else:
            print(f"Secuencia {i+1} generada de {num_sequences}")
    return pd.DataFrame({'sequences':sequences})

In [26]:
generated_seqs = generate_multiple_sequences(
    num_sequences=3000,
    max_new_tokens=50,
    temperature=2.0,
    prompt = 'A A V A L L P A V L L A L L A P Q L G K K K H R R R P S K K K R H W',
    repetition_penalty = 5.0,
    show_output=False

)

Secuencia 1 generada de 3000
Secuencia 2 generada de 3000
Secuencia 3 generada de 3000
Secuencia 4 generada de 3000
Secuencia 5 generada de 3000
Secuencia 6 generada de 3000
Secuencia 7 generada de 3000
Secuencia 8 generada de 3000
Secuencia 9 generada de 3000
Secuencia 10 generada de 3000
Secuencia 11 generada de 3000
Secuencia 12 generada de 3000
Secuencia 13 generada de 3000
Secuencia 14 generada de 3000
Secuencia 15 generada de 3000
Secuencia 16 generada de 3000
Secuencia 17 generada de 3000
Secuencia 18 generada de 3000
Secuencia 19 generada de 3000
Secuencia 20 generada de 3000
Secuencia 21 generada de 3000
Secuencia 22 generada de 3000
Secuencia 23 generada de 3000
Secuencia 24 generada de 3000
Secuencia 25 generada de 3000
Secuencia 26 generada de 3000
Secuencia 27 generada de 3000
Secuencia 28 generada de 3000
Secuencia 29 generada de 3000
Secuencia 30 generada de 3000
Secuencia 31 generada de 3000
Secuencia 32 generada de 3000
Secuencia 33 generada de 3000
Secuencia 34 genera

In [ ]:
generated_seqs.to_csv(out_dir)